In [58]:
import requests
import json
from datetime import datetime, timedelta

import pandas as pd
import cx_Oracle
from flask import Flask, request
from flask_restful import Resource, Api
from flask import Response
import asyncio
import websockets
import aioconsole

In [59]:
#검색 함수
def search(word,x):
    a=str(x)
    rs=a.find(word)
    if rs == -1:
        return False
    else:
        return True

In [60]:
# rains의 모든 컬럼을 갖고오는 함수( 나중에 수정예정)
def select_from_oracle():
    con = cx_Oracle.connect('project_thejoeun/1541@localhost:1521/xe')
    cur = con.cursor()
    sql_select = '''
            select * from rains
             '''
    rs = cur.execute(sql_select)

    data=[]
    for record in rs:
        row=[]
        for source in range(0, len(record)):
            row.append(record[source])
        data.append(row)




    con.commit()
    cur.close()
    con.close()
    return data


In [61]:
# festival의 모든 컬럼을 갖고오는 함수( 나중에 수정예정)
def select_from_oracle2():
    con = cx_Oracle.connect('project_thejoeun/1541@localhost:1521/xe')
    cur = con.cursor()

    sql_select = '''
            select * from festival
             '''
    rs = cur.execute(sql_select)
    data=[]
    for record in rs:
        row=[]
        for source in range(0, len(record)):
            row.append(record[source])
        data.append(row)

    con.commit()
    cur.close()
    con.close()
    return data


In [62]:
#함수를 사용해 불러오기
data=select_from_oracle()

In [63]:
column_list_ra=['baseDate', 'baseTime' ,'categorys'  ,'fcstDate' ,'fcstTime' ,'fcstValue'  ,'nx'   ,'ny']

In [64]:
ra_df = pd.DataFrame(data, columns=column_list_ra)

In [65]:
ra_df

,baseDate,baseTime,categorys,fcstDate,fcstTime,fcstValue,nx,ny
0,2023-07-18,1100,TMP,2023-07-18,1200,23,37,126
1,2023-07-18,1100,POP,2023-07-18,1200,30,37,126
2,2023-07-18,1100,PCP,2023-07-18,1200,0,37,126
3,2023-07-18,1100,TMP,2023-07-18,1300,23,37,126
4,2023-07-18,1100,POP,2023-07-18,1300,30,37,126
...,...,...,...,...,...,...,...,...
1093,2023-07-18,1100,POP,2023-07-20,2300,0,37,126
1094,2023-07-18,1100,PCP,2023-07-20,2300,0,37,126
1095,2023-07-18,1100,TMP,2023-07-21,0000,25,37,126
1096,2023-07-18,1100,POP,2023-07-21,0000,0,37,126


In [66]:
column_list_fa=['f_name','place','open_date','close_date','info','host_or','PHONE_NUMBER','ADRESS','LATITUDE','HARDNESS']

In [67]:
data2=select_from_oracle2()

In [68]:
fa_df = pd.DataFrame(data2, columns=column_list_fa)

In [69]:
fa_df

,f_name,place,open_date,close_date,info,host_or,PHONE_NUMBER,ADRESS,LATITUDE,HARDNESS
0,울산옹기축제,온라인 축제,2022-09-30,2022-10-03,국내+유일의+옹기특화+축제++온라인비대면축제,울산광역시 울주군청 관광과,052-204-0355,울산광역시 울주군 온양읍 외고산2길 2-13,35.432991,129.278447
1,금산인삼축제,금산인삼관 광장 및 인삼약초시장 일원,2022-09-30,2022-10-23,ㅇ인삼캐기+체험+인삼약초체험+생활건강+체험+등+건강+체험관+운영+국제인삼교역전+인삼...,충청남도 금산군,041-750-4144,충청남도 금산군 금산읍 인삼광장로 30,36.100026,127.500730
2,횡성한우축제,횡성종합운동장,2022-09-30,2022-10-04,한우구이터+횡성한우+발전포럼+횡성빛축제+등,강원도 횡성군청,033-340-5712,강원도 횡성군 횡성읍 문화체육로 57,37.496087,127.989150
3,원주다이내믹댄싱카니발,댄싱공연장 및 원일로 일원,2022-10-01,2022-10-03,댄싱경연++문화예술공연++각종+체험+등,강원도 원주시청+제36사단,033-737-5132,강원도 원주시 단구로 170,37.336430,127.946261
4,소래포구축제,인천 남동구 소래포구 일원,2022-10-01,2022-10-03,관광축제,인천광역시 남동구,032-453-2142,인천광역시 남동구 아암대로 1454,37.387252,126.726980
...,...,...,...,...,...,...,...,...,...,...
1176,2023년 은계호수공원 가을콘서트,은계호수공원,2023-10-07,2023-10-08,시흥예총,경기도 시흥시청,031-310-6731,경기도 시흥시 계수동 716,37.445256,126.806582
1177,동두천 무형문화재 축제,소요산 축산물브랜드 육타운 상설무대,2022-10-29,2022-10-29,동두천+무형문화재+축제,경기도 동두천시무형문화재연합회,031-860-2064,경기도 동두천시 상봉암동 산 1-5,37.938776,127.085743
1178,왕방산 국제MTB대회,왕방산 일원,2022-10-21,2022-10-21,국제MTB대회,(사)대한자전거연맹,031-860-2209,경기도 동두천시 어등로 45,37.900840,127.070499
1179,천사마라톤대회,종합운동장,2022-10-03,2022-10-03,천사데이기념+마라톤대회,사)희망지킴이 천사운동본부,031-861-1004,경기도 동두천시 어등로 45,37.900840,127.070499


# 아래 l_name => 입력할것 넣으면 검색가능

In [70]:
l_name='은계호수공원'

In [72]:
list_lo=fa_df[fa_df['place'].apply(lambda x: search(l_name,x))]
list_lo

,f_name,place,open_date,close_date,info,host_or,PHONE_NUMBER,ADRESS,LATITUDE,HARDNESS
1176,2023년 은계호수공원 가을콘서트,은계호수공원,2023-10-07,2023-10-08,시흥예총,경기도 시흥시청,031-310-6731,경기도 시흥시 계수동 716,37.445256,126.806582


In [75]:
def search_name_la_ha(name):
    list_lo=fa_df[fa_df['place'].apply(lambda x: search(l_name,x))]
    x=list_lo['LATITUDE']
    y=list_lo['HARDNESS']
    return x, y

In [76]:
x,y=search_name_la_ha(l_name)

In [77]:
x=int(x)
x

C:\Users\tj-bu\AppData\Local\Temp\ipykernel_17500\3718533679.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  x=int(x)


37

In [78]:
y=int(y)
y

C:\Users\tj-bu\AppData\Local\Temp\ipykernel_17500\4206905778.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  y=int(y)


126

In [85]:
def search_ra(x, y):
    lst=ra_df[(ra_df['nx']==x) & (ra_df['ny']==y)]
    return lst

In [86]:
z=search_ra(x,y)

In [87]:
z

,baseDate,baseTime,categorys,fcstDate,fcstTime,fcstValue,nx,ny
0,2023-07-18,1100,TMP,2023-07-18,1200,23,37,126
1,2023-07-18,1100,POP,2023-07-18,1200,30,37,126
2,2023-07-18,1100,PCP,2023-07-18,1200,0,37,126
3,2023-07-18,1100,TMP,2023-07-18,1300,23,37,126
4,2023-07-18,1100,POP,2023-07-18,1300,30,37,126
...,...,...,...,...,...,...,...,...
1093,2023-07-18,1100,POP,2023-07-20,2300,0,37,126
1094,2023-07-18,1100,PCP,2023-07-20,2300,0,37,126
1095,2023-07-18,1100,TMP,2023-07-21,0000,25,37,126
1096,2023-07-18,1100,POP,2023-07-21,0000,0,37,126
